In [1]:
from keras.models import load_model
from pkg_resources import resource_filename
from utils2 import one_hot_encode
import numpy as np

import datetime as dt
import h5py
import numpy as np
import os
import fnmatch
import time
import re
from sklearn.metrics import average_precision_score
from pandas import DataFrame
import pandas as pd

In [2]:
import sys
sys.path.append('/home/ychwang/jklab-master')
import jkgenome

jkgenome import 할 수 있게 경로를 추가해주세요

In [3]:
import_test=jkgenome.locus('chr8:140300606-140300626-')
import_test.twoBitFrag2('hg38').upper()

'TTGAAACAACAGGCATGTGA'

In [31]:
import_test=jkgenome.locus('chr8:140300606-140300626+')
import_test.twoBitFrag2('hg38').upper()

'TCACATGCCTGTTGTTTCAA'

In [4]:
paths = ('reg_categorical_softmax.h5' for x in range(1))
model_reg=[load_model(resource_filename('spliceai', x),compile=False) for x in paths]

In [5]:
paths = ('SpliceAI400_g{}.h5'.format(x) for x in [1,2,3,4,5])
model_default=[load_model(resource_filename('spliceai', x),compile=False) for x in paths]

In [6]:
paths = ('gtex_model/SpliceAI10000_g{}.h5'.format(x) for x in [1,2,3,4,5])
baseline=[load_model(resource_filename('spliceai', x),compile=False) for x in paths]

In [63]:
def predict_sequence_with_context2(dna_sequence,strand,model,assembly='hg38'):
    encoded_dna_sequence = one_hot_encode(dna_sequence)[None, :]
    
    if strand == '-':
        encoded_dna_sequence = encoded_dna_sequence[:, ::-1, ::-1]
    

    predicted_value = np.mean([model[m].predict(encoded_dna_sequence) for m in range(len(model))], axis=0)
    
    
    acceptor_prob = predicted_value[0, :, 1]
    donor_prob = predicted_value[0, :, 2]

    acceptor_prob_sorted_index=acceptor_prob.argsort()
    acceptor_prob_sorted_index=np.flip(acceptor_prob_sorted_index)
    
    donor_prob_sorted_index=donor_prob.argsort()
    donor_prob_sorted_index=np.flip(donor_prob_sorted_index)
    
    acceptor_prob_sorted_value=acceptor_prob[acceptor_prob_sorted_index]
    donor_prob_sorted_value=donor_prob[donor_prob_sorted_index]
    
    #                 0          1             2                 3                          4
    return predicted_value, acceptor_prob, donor_prob, acceptor_prob_sorted_index, donor_prob_sorted_index,\
        acceptor_prob_sorted_value, donor_prob_sorted_value

In [ ]:
new_spliceAI()

In [7]:
context=5000
input_sequence = 'AGGATTCCCTATTTAATAAATGGTGCTGGGAAAACTGGCTAGCCATATGTAGAAAGCTGAAACTGGATCCCTTCCTTACACCTGATAGAAAAATCAATTCAAGATGGATTAAAGACTTAAACATTAGACCTAAAACCATAAAAACCCTAGAAGAAAACCTAGGCATTACCATTCAGGACATAGGCATGGGCAAGGACTTCATGTCTAAAACACCAAAAGCAATGGCAACAAAAGACAGAATTGACAAATGGGATCTAATTAAACTAAAGAGCTTCTGCACAGCAAAAGAAACTACCATCAGAGTGAACAGGCAACCTACAAAATGGGAGAAAATTTTCGCAACCTACTCATCTGACAAAGGGCTAATATCCAGAATCTACAATGAACTCAAACAAATTTACAAGAAAAAAACAAACAACCCCATCAAAAAGTGGGCGAAGGACATGAACAGACACTTCTCAAAAGAAGACATTTATGCAGCCAAAAAACACATGAAAAAATGCTCATCATCACTGGCCATCAGAGAAATGCAAATCAAAACCACAATGAGATACCATCTCACACCAGTTAGAATGGCAGTCATTAAAAAGTCAGGAAACAACAGGTGCTGGAGAGGATGTGGAGAAATAGGAATCCTTTTACACTGTTGGTGGGACTGTAAACTAGTTCAACCATTGTGGAAGTCAGTGTGGTGATTCCTCAGGGATCTAGAACTGGAAATACCATTTGACCCAGCCATCCCATTACTGGGTATATACCCAAAGGACTATAAATCATGCCGCTATAAAGACACATGCACACGTATGTTTATTGTGGCATTATTCACAATAGCAAAGACTTGGAACCAACCCAAATGTCCAACAATGATAGACTGGATTAAGAAACTGTGGCACATATACACCATGGAATACTATGCAGCCATAAAAAATGATGAGTTCATGTCCTTTGTTAGGGACATGGATGAAATTGGAAATCATCATTCTCAGTAAACTGTCGCAAGAAGAAAAAACCAAACACCGCATGTTCTCACTCATAGGTGGGAATTGAACAATGAGATCACATGGACACAGGAAGGGGAATATCACACTCTGTGGACTGTTGTGGGGTGGGGGGAGGGGGGAGGGATAGCATTGGGAGATATACCTAATGCTAGATGACGAGTTAGTGGGTGCAGCACACCAGCATGGCACATGTATACATATGTAACTAACCTGCACAATGTGCACATGTACCCTAAAACTTAAAGTGTAATAAAAAAAATAATAATTTTTAAAGTTTGCATATTATTTTGTTATGTAGACATATTACAGGTTATATATCCATTTTACTATTGTTGGACATTTAGTAGAAAAAGTACTGTAGTTTTCATACTTACACATGAACCTTGAATTGTTTGATTAGGTAAATTTTTTTTTTTTTTTTTTTTTTGAGACAGAGTCTCGCTTCGTCAAACCCAGGCTGGAGTGCAGTGGCATGATCTCAGCTCACTGCAATCTCTGCCTCCTGAGTTCAAGTGATTCTCCTGCCTCAGCTTCCTGAGTAGCTGGGATTACAGGTGCCCACCACCACACCCAGCTAATTTTTGTATTTTAAATAGAGACAAGGTTTCACCATGTTGGCCAGGCTGGTTTCGAACTCCCGACCTCAGGTGATCCACCTGGCTCTGCCTCCCAAATTGCTGAGATTACAGATGTGAGCCACTGTGCCCAGCCTGATTAGGTAAATTTTGACTACAGCATGCTCCTGCAAGAAGCCATCTTGAACATCTTTGTTTCTCTTCCTTGAAGGCATCCTTCATCAAAAAGCCATTTGACCGTGGAGAAGTAGAATCAATGGAAGATGATACTAATGGAAATCTAATGGAGGTGGAGGATCAGTCATCCATGAATCTATTTAACGATTACCCTGATAGTAGTGTTAGTGATGCAAACGAACCTGGAGAGAGCCAAAGTACCATAGGTAAATACATATTTACTACTTGGGATTTCTTTTACTTCTTTATATTGATTTGGCAGTATAAGAGGCCTCATTGATATCAATTTTGTGCTTATTTCATTTTCTCTTAGTATAGCCTTTTAGGATTGTTCCTTTCTTATATACTTTATTTTTTTTTTATTTTTACTTGAATTTATTAGTTTCATATTTTATTCTTCATAGAAGGAACTTAAGATAACTATTAAAGAAATAAAACCAGGCCAGGCATGGTGGCTCACGCTTGTAATCTCAGCACTTTGGGAGGCCAAGGCGGGCTGATCATGAGGTCAGGAGATCAAGACCATCCTGGCTAACGTGGTGAAACCCCATCTCTACTAAAAACACAAAAATTAGATGGGCGTGGTGGCGGGCGCCTGTAGTCCCAGCTACTCGGGAGGCTGAAGCAGGAGAACGGCATGAACCTGGGGAGGTGGAGGTTGCAGTGAGCCGAGATCATGCCACTGCATTCCAGCCTGGGCGACAGAGCGAGACTCCGTCTCAAAAAAAGAAATAAAATCATATGATAGGTTATTTCCTCAATAGAAAGCAGGAAATGAAAACACACCTGTAATGCCATTGCTTTTAACCTTTTTCTATTATTTCTCTGTAGAAACATTCAAAATTGTAGTCAACATCAACATTCACATGTTATGAATGCCTGTTTAATTATAAGTATTTTTCCAGGTAGTTGCTTTTCTTGATAGTCATACTTTTTAATGGCTTCACCATTTTTCTGTCATAAATACCATAATTTACTTGACATTTTTTATTGTCAGAAATTTAGATTCTTTTCTATTTTCCTACAATAAACATTATTGCAATGATCATTTGTCTAAGGGTTTTTTTTTCTCATATTTTAAATTATTTCTTGACAACAGAATCTTGGATAATTTTTCAAAAAGACTTTTGAAGCTTTCAGTATATAATTAATTTCACTATAATTTTGCTTTTCATATACTTTTTTTTGTGAAGAGGAGGAAATTTGAGTTAATATGACTATATATGGCTGTTGTGCCCTTCTCTTAGTGTTAATGAGTGCTTTTTATTTTTAGGTGCCATTAATCCTTTAGCTGAAGAATATCTGTCAAAGCAAGATCTACTTTTCTTAGACATGCTCAAGTTCTTGTGTTTGTGTGTAACTACTGCTCAGACCAATACTGTGTCCTTTAGGGCAGCTGATATTCGGAGGAAATTGTTAATGTTAATTGATTCTAGCACGCTAGAACCTACCAAATCCCTCCACCTGCATATGGTGAGTTACGTTAAATGAAGAAGCTCTTGGATTTTATCTGATGTTGCTGACTAAATGTAATGAGTTGACATGTAAGAATCACATGGTGTCTTTGAAGAATTGAAATTGCTTTCTTGAGAAATGAACCTGAGACTAGTTGGAAAATAACACTTTTAACGTGCTGTGAGCAAATTTAAGTGGATGCTGAAATATTAAAACTTAGATGGGTTTTGGATAGATTTATTCTCTTTAGGTTTAATTGTTATAAATTATTTTAATTAGGCTAAGAAACATAGACCCTGAATGTAAAAATATTACATAACATTGTGTTTGAAGTTGGAACCTTGCCTTTGACCTTAGTTTATATATTTTCAGTTTTTTGGATTTGATTTCAAAATGATAGTGAAGTTACAAGAATACTACAGTTAACGTTTTACCATATTTGCTTAATCATTCCCTCTCTCCACACACACACTCATACGCAGATACACACATCATTTTCTCTGAACCATTTTGAGATGAAATCGCACACGTGATGCTCCTTATACTTCAGTGTATATTTCCTAAAAACAAGGATACACTTCTGTTTAGCCACAGTATAATCATCAAAATCAGGAAGTTAACATTTTGTCATTGCATTTAGGTGTGTAGTCTGTTTAGTCTCCTTCAATCTGGAACAGTTCTTTTTTTTATTTATCATGACCTTCATATATTTGAATAGTACAGGTCCCTTGCTCTGAGTTTGTCTGATGTTTCCTTATGATTAAATTTATACTTTGACGAGAATGTTGCAAAAGTAATTCCTCTTCTGCCTTCACCCTGTCCATTTATCTGTCTGTTGATAGATATCTTCAAATCCAAGCCAACATCACACAGTTTATCTTTTTTTCTGTCCAAATTTATAAATTGCTTTACTAGCAGTTAAAAAAAACACCCCAAAACCTGACTATCCACTCAACCTGAATGCATTTACTTATTTGCTCAACAACCTTATATATAACTAATCTAACCATGTGGGCCATTTTCTTAGCCCTAGCCCCAGTGTATGTGGAGGTAGAGCCTCTGGCCTAGCGAGTAGCAAGTTGGGTCCTAGGTCATGTCCCCAGTTCCAGCAAATTGTCACACATGCCTGGTGAGTCCCTAATTAGAACCTCTGATCACAGTAAATTGTAGGCTGAGTCATTGGTGGAATCCCTGGTCATGTTTCCCACCCTTGGTGAAAATGCACGTCAATCCCTTAGAGTGAGAAGACAAGTTTGGTTGTGCCCTGTTTGGGCCCCTGGTTTAGGTCCTCGCAGATACTCAAGTCTCTATCCCTGACCAAGAGGAAGTTACATGCTTTTCTTGAAGTATTGATGTTTCAAGTCACTTCAGAACATTCAGTAACCTCATGTACATGACTCTCATGTCAAATAATGAAAAGGATCATTTAATTTCCTTTGTTAATATCTGAAAAGGATTAAGAAATATATTATTCTATATGTAAATAGGAAAAAAAATCCCTTATTATTTCAGTATAGCCAGAAAATTATTTGGTAGTTCTCGGAAACTGGCTGCCCAGATAATAACTACCATTATAACTGGTCGTTGCAGCAGCCCTTTCTGTGCATAGTACCATAAAGAGCCTTACAAGATTGCCTTATGGGAAGATAGGTCAAGGTTTTTGACTTCTTCCCTGCCCATGATTATCTCTGCTTTCCAAAATTCATTAATAGTTTTCAACCACTGGTCTATTATCCCCAAGCATTATTTTGGGAGAGTGGAGACTTACAGTTTCAGAATCTTGCTCAAGCTCTTAACTGCAACAGTGGTAATAATGATCATTTATTGAATTCCACAATAGAAGCTAGACTTTTACGTTTATTTTCTCTAATCCTCACAGTTATCTGGCCAGGTAAGTGATATATCTTCACTCTACTGATGAGGGTACGAAGGCCCTAGATGACATAAGGCAAGTTTTTTACCAGAAAAGTTAAGGCTGTGGTACTGGGATTTGAATTTTCTGTTTGACTTCAAAGCCTTTTGTTTTTACTATACACCTCTTAAATATTTACAGCTGTGTGTGTGTGTGTATGTGTGTGCATGTATGAATTTATGTTTATATACTTAAAAAAAATTTTTTTTTGAGACGGAATCTCGCTCTGTTGCCCGGGCTGGAGTGCAGTGGCGCGATCTCGGCTCACTGCAACCTTCGCCTCCCAGGTTCAAGCGATTCTCCTGCCTCAGCCTCCTGAGTAGCTGGGATTATAGGCGCCTGCCACCACCCCTGGCTAATTTTTGTGTTTTTAGTAGAGATGGAGTTTCACCATATTGGCCAGGCTGTTCTCAAACTCCTGACCTTGTGATCTGCCTGCTTCAGCCTCCCAAAGTGCTGGGATTACAGGTGTGAGCCACTGCACCCGGCCTATGTTTATATACTTTTTAAAGTAAATGATTTGTGGATAAACCTGATTTTTTTCCCTCCTACCATCTTAGTATCTAATGCTTTTAAAGGAGCTTCCTGGAGAAGAGTACCCCTTGCCAATGGAAGATGTTCTTGAACTTCTGAAACCACTATCGTAAGAAATTAAAACCTTATGTTATGTTCACTTTAAAGTTATAAAATAACTGATGTGTTCTGTTAAGCTTATAAAGTTGAACTTTTTTTTTTTTTTTACCACAGCAATGTGTGTTCTTTGTATCGTCGTGACCAAGATGTTTGTAAAACTATTTTAAACCATGTCCTTCATGTAGTGAAAAACCTAGGTCAAAGCAATATGGACTCTGAGAACACAAGGGATGCTCAAGGACAGTTTCTTACAGTAATTGGAGCATTTTGGTAGGTACAGTCTATTTTGTGGTCCTATTTTTCTTTTGCTATCTGTGGATACGAATGCAAGTTTTGTATCCACATCAGTGATTTCTTCTGATCTTCCTACATAGCTAATACATCTTTTAAGAATAGCAGAATGTAATTTGTGTTTCCCTCAGTCGCTTGAAGAACTACATTGCTTTTTGTTTAAGGCTTGGCTTTCTAAACCGTTTAACTCATTTCTCTGCAAATTTTGAATCACTTAATAATTTGACATGTATAGTGAATAAAGGATAACAAGCCAAAATCAGTTGAATTGTTCATGTTCCCTAAGTGAACAAATTGGCCTTGTCCTTATCTGGTTATTTTATGCATATTGTCACATTCCAAGATAACACCCTGATACTTACAGAGGAGCACATACATATAATTTAGAACCCTTGTACTATCTGATTGCTAAGTAATTCTAAATCAATAGACTGTTTTTTGTTTTTGTGTTTTTTTGAGATGGAGTCTCACTCCATCACCCAGGCTGCAGGCTGGAGTGCAGTGGTGCTATATCTGCTCACTGCAACCTCCGCCTAGTAGGTTCAAGCAATTGTCCTGCCTCAGCCTCCTGAGCAGCTATGATTACAGGCTCCCACCACAATGCCTGGCTAATTTTTTTGTATTTTTAGTAGAGATGGGGTTTCAACACGTTGGCCAGGCTGGTCCCAAACTCCTGACCTCAAGTGATCTGCCCACCTTGGCCTCCCAAAGTGTTGGGATTACAGGTGTGAGCCAGTGCACCCAGCCTGAATGTGGTTTTGAAATCTTCAATATACCAAACAAAGAAATCTTCAATATACCAAACAAAGAAATTTCTTTCTAAAGTAAGTTTACTGTAATGTAGTTTAGCCATTCTATGGTAGCCCCCAAAAAAGGACATAATGGTATAAAACAAATAATATACGCTAAAATGAATTCTTTTACACTAATTTCTTTTAGCTTGAATTTTTGGCAAGGTGAGTATGTTGGCATATTCCACATAATGACAAATAAGTTTAGCACAGAAAGACATATTGGAAGTAACTTACAATAACCTTTCAGTGAGTTTTCTGAGTGCTTTTATCAGAATGATTATTTAACTTTGGAAAACTTACTTGATTTCAGGCATCTAACAAAGGAGAGGAAATATATATTCTCTGTAAGAATGGCCCTAGTAAATTGCCTTAAAACTTTGCTTGAGGTGAGTTTTTGCATTTTTTTAGTAAGATCTCCATTGAAAATTTTAAAGCAGTCTTTGTTTGTTAATGAGTAATTTTTCTCTATTTCATATTTAACCACAGTTCTTTTCCCGTAGGCTGATCCTTATTCAAAATGGGCCATTCTTAATGTAATGGGAAAAGACTTTCCTGTAAATGAAGTATTTACACAATTTCTTGCTGACAATCATCACCAAGTTCGCATGTTGGCTGCAGAGTCAATCAATAGGTAATGGGTCAAATATTCATGAAGTATTTGGAATGCTGCAGATGGCAGTAGAATGTCTTACATAGTAACAGCTCACAGTTGCAATATTAAAAATAGCTAACACTTGTTGAGTATATACGGTGTGCCTGGCATTTATGTTTATTCTTAATTCTTATACTTCTGTCACTTAGATTCTATTATTTCCTTCAATTTATAAATGAAAACTGTGACTTAGTGAGGTAAAGTAACTTGCTTATGCCACAGAGCTAACAAGTGGTTGACTCGAGTGAGGTGTGTCTACTATCATGTACAGGAGATACGGACTGTTTGGAAATTACTGTGATTTTATGTGAGTTTATGAAATGATAATTAGCACATTTTACATTGTGCTAATTACAATTATATTTACTTGAGTGAACATCCAGTGAAGCACTGGAAGGATTCTATCATGAGGATCTTTTATATTTGAGTACTGGAATAAACATATATTAATTTCATTCTTTTTTTTTTTTTTTTTTTTTTTTTTTGAGATGGAGTTTCGCTCATGTCTCCCAGGCTGGAGTGCAATGGTGTGATCTCGGTTCACTTCAGCCTCTACCTCCTCGGTTCAAGCAATTCTCCTGCCTCATCCTCCTGAGTAGCTGGGATTACAGGCGCCTGCCACCATGCCCAGCTAATTTTTTTATTTTTCGTAGAGATGGAGTTTCACCATGTTGGCCAGGCTGGTCTTGAACTCCTGACCTCAGGTGATCCACCCACCTTGTCCTCCCAAAGTGCTGAGATTACAGGCGTGAGCCTTTAATCATGTAGTTTTTGTCATTCGTTCTGTTTATGTGATGGATTATGTTTATTGATTTGTGTATGCTGAATCAGCCTTGCGTCCCAGGGATGAAGCTGACTTGATAGTGGTGGATAAGCGTTTTGATGTGCTGGATTCAGTTTGCCAGTATTTTATCGAGGATTTTCACATCGATGTTCATCACGGATAATGGCCTGAAATTTTCTTTTTCTTGTGTCTCTGCCAGGTTTTGGAATCAGGATGATGCTGGCATCATAAAATGAGTTAGGGAAGAGTCCCTCTTTTTCTATTGTTTGGAATAGTTTCAGAAGGAATGGTACCAGCTCCTCTTTGTACCCCTGGTAGAATTCGGCTGTGAATCCCTCTGGTCCTGGGCTTTTTTTGGTTGGTAGGCTATTAATTACTGCCTCAATTTCAGAACTTGTTATTGGTCTATTCAGGGATTCGACTTCTTCCTGCTTTAGTTGGGAGGTTGTATGTGTCTAGGAATTTATCACTTTCTTCTAGATTTTCTAGTTTATTTGCATAGAGGTATTTATAGTATTCTCTGATGGTAGTTTGTATTTCTGTGGGATCAGTGGTATTATCCTCTTTATCATTTTTTATTGAGTCTATTTGATTCTTCTCTCTTTATTAGTCTTGCTAGCGGTCTGTTTTGTTAATCTTTTCAAAAAATCAGCTCCTGGATTAATTGATTTTTTTGAAAGGTTTTTTTTGTGTGTGTGTGTCTCTATCTCCTTCACTCAGCTCTGATCTTAGTTATTTCTTGTCTTCTGCTAGCTTTTGAATTTGTTTGCTCTTGCTTCTCTAGTTCTTTTAATTGTGATGTTAGGGTGTTGATTTTAGATCTTTCCCACTTTGTCCTGTGGGCATTTAGTGCTATAAATTTCCCTCTAAACACTGCTTTAGCTGTGTCCCAGAGATTCTGGTACATTGTGTCTTTGTTCTTATTGGTTTCAAAGAACTTACTTATTTCTGCCTTCATTTCATTATTTACCTAGTAGTCATTCAGGAGCAGGTTGTTCAGTTTCCATGTAGTTGTGTGGTTTTGAGTGAGTTTCTTAATCCTCAGTTCTAATTTGATTGCACTGTGATCTGAGAGACTGTTACGATTTCCATTCTTTTGCATTTGCTGAGGTTTGTTTTACTTTCAATTATGTGGTCAGTTTTAGAATAAGTGTGATGTGGTGCTGAGAAGAATGTATATTACTCTGTTGATTTGGGGTGGAGAGTTCTGTAGATGTCTGTTAGGTCTGCTTGGTCCAGAGTTCGGTTCAAGTCCTGAATATCCTTGTTAATTTTCTGTCTCATTGATCTGTCTAATATTGATGGTGGGGTGTTAAAGTCTCCCGCTATTATTGTGTGGAAGTCTAAGTCTCTTCGTAGGTCTAAGAACTTGCTTTATGAATCTGGGTGCTCCTGTATTGGGTGCATATATATTTAGGATAGTTAGCACTTCTTGTTGCATTGATCCTTATACCATCATGTAATGTCCTTCTTTGTCTTTCTTGATCTTTGTTGGATGTTTGTTTTATCAGAGACTAGGATTGCAACCCTTGCTTTTTTTCTTTCTATTTGCTTGGTAAATATTCCTCCACCCCTTTATTTTGAGCCTATGTGTGTCTTTGCATGTGAGATGGGTCCCCTGAATACAGCACACTGATGGGTCTTGACTCTTTATCCAATTTGCCAGTCTCTGTCTTTTAACTGGGACATTTTAGCCCATTTACATTTAAGGTTAATATTGTTATGTGTGAATTTGATCCTGTCATTATGATGCTAACTGGTTATTTTGCCCATTAGTTGATGCAGTTTCTTCATAGTGTCAATGGTCTTTACAATTTGGTATGTTTTTGCAGTGGCTGGTACCAGCTTTGCCTTTCCATATTTAGTGCTACCTTCAGGAGCTCTTGTAAGGCAGGCCTGGTGGTGACAAAATCTCTCAGCATTTGCT'

x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
y = np.mean([baseline[m].predict(x) for m in range(5)], axis=0)

p1 = y[0, :, 1]
p2 = y[0, :, 2]

In [8]:
context=5000
input_sequence = 'AGGATTCCCTATTTAATAAATGGTGCTGGGAAAACTGGCTAGCCATATGTAGAAAGCTGAAACTGGATCCCTTCCTTACACCTGATAGAAAAATCAATTCAAGATGGATTAAAGACTTAAACATTAGACCTAAAACCATAAAAACCCTAGAAGAAAACCTAGGCATTACCATTCAGGACATAGGCATGGGCAAGGACTTCATGTCTAAAACACCAAAAGCAATGGCAACAAAAGACAGAATTGACAAATGGGATCTAATTAAACTAAAGAGCTTCTGCACAGCAAAAGAAACTACCATCAGAGTGAACAGGCAACCTACAAAATGGGAGAAAATTTTCGCAACCTACTCATCTGACAAAGGGCTAATATCCAGAATCTACAATGAACTCAAACAAATTTACAAGAAAAAAACAAACAACCCCATCAAAAAGTGGGCGAAGGACATGAACAGACACTTCTCAAAAGAAGACATTTATGCAGCCAAAAAACACATGAAAAAATGCTCATCATCACTGGCCATCAGAGAAATGCAAATCAAAACCACAATGAGATACCATCTCACACCAGTTAGAATGGCAGTCATTAAAAAGTCAGGAAACAACAGGTGCTGGAGAGGATGTGGAGAAATAGGAATCCTTTTACACTGTTGGTGGGACTGTAAACTAGTTCAACCATTGTGGAAGTCAGTGTGGTGATTCCTCAGGGATCTAGAACTGGAAATACCATTTGACCCAGCCATCCCATTACTGGGTATATACCCAAAGGACTATAAATCATGCCGCTATAAAGACACATGCACACGTATGTTTATTGTGGCATTATTCACAATAGCAAAGACTTGGAACCAACCCAAATGTCCAACAATGATAGACTGGATTAAGAAACTGTGGCACATATACACCATGGAATACTATGCAGCCATAAAAAATGATGAGTTCATGTCCTTTGTTAGGGACATGGATGAAATTGGAAATCATCATTCTCAGTAAACTGTCGCAAGAAGAAAAAACCAAACACCGCATGTTCTCACTCATAGGTGGGAATTGAACAATGAGATCACATGGACACAGGAAGGGGAATATCACACTCTGTGGACTGTTGTGGGGTGGGGGGAGGGGGGAGGGATAGCATTGGGAGATATACCTAATGCTAGATGACGAGTTAGTGGGTGCAGCACACCAGCATGGCACATGTATACATATGTAACTAACCTGCACAATGTGCACATGTACCCTAAAACTTAAAGTGTAATAAAAAAAATAATAATTTTTAAAGTTTGCATATTATTTTGTTATGTAGACATATTACAGGTTATATATCCATTTTACTATTGTTGGACATTTAGTAGAAAAAGTACTGTAGTTTTCATACTTACACATGAACCTTGAATTGTTTGATTAGGTAAATTTTTTTTTTTTTTTTTTTTTTGAGACAGAGTCTCGCTTCGTCAAACCCAGGCTGGAGTGCAGTGGCATGATCTCAGCTCACTGCAATCTCTGCCTCCTGAGTTCAAGTGATTCTCCTGCCTCAGCTTCCTGAGTAGCTGGGATTACAGGTGCCCACCACCACACCCAGCTAATTTTTGTATTTTAAATAGAGACAAGGTTTCACCATGTTGGCCAGGCTGGTTTCGAACTCCCGACCTCAGGTGATCCACCTGGCTCTGCCTCCCAAATTGCTGAGATTACAGATGTGAGCCACTGTGCCCAGCCTGATTAGGTAAATTTTGACTACAGCATGCTCCTGCAAGAAGCCATCTTGAACATCTTTGTTTCTCTTCCTTGAAGGCATCCTTCATCAAAAAGCCATTTGACCGTGGAGAAGTAGAATCAATGGAAGATGATACTAATGGAAATCTAATGGAGGTGGAGGATCAGTCATCCATGAATCTATTTAACGATTACCCTGATAGTAGTGTTAGTGATGCAAACGAACCTGGAGAGAGCCAAAGTACCATAGGTAAATACATATTTACTACTTGGGATTTCTTTTACTTCTTTATATTGATTTGGCAGTATAAGAGGCCTCATTGATATCAATTTTGTGCTTATTTCATTTTCTCTTAGTATAGCCTTTTAGGATTGTTCCTTTCTTATATACTTTATTTTTTTTTTATTTTTACTTGAATTTATTAGTTTCATATTTTATTCTTCATAGAAGGAACTTAAGATAACTATTAAAGAAATAAAACCAGGCCAGGCATGGTGGCTCACGCTTGTAATCTCAGCACTTTGGGAGGCCAAGGCGGGCTGATCATGAGGTCAGGAGATCAAGACCATCCTGGCTAACGTGGTGAAACCCCATCTCTACTAAAAACACAAAAATTAGATGGGCGTGGTGGCGGGCGCCTGTAGTCCCAGCTACTCGGGAGGCTGAAGCAGGAGAACGGCATGAACCTGGGGAGGTGGAGGTTGCAGTGAGCCGAGATCATGCCACTGCATTCCAGCCTGGGCGACAGAGCGAGACTCCGTCTCAAAAAAAGAAATAAAATCATATGATAGGTTATTTCCTCAATAGAAAGCAGGAAATGAAAACACACCTGTAATGCCATTGCTTTTAACCTTTTTCTATTATTTCTCTGTAGAAACATTCAAAATTGTAGTCAACATCAACATTCACATGTTATGAATGCCTGTTTAATTATAAGTATTTTTCCAGGTAGTTGCTTTTCTTGATAGTCATACTTTTTAATGGCTTCACCATTTTTCTGTCATAAATACCATAATTTACTTGACATTTTTTATTGTCAGAAATTTAGATTCTTTTCTATTTTCCTACAATAAACATTATTGCAATGATCATTTGTCTAAGGGTTTTTTTTTCTCATATTTTAAATTATTTCTTGACAACAGAATCTTGGATAATTTTTCAAAAAGACTTTTGAAGCTTTCAGTATATAATTAATTTCACTATAATTTTGCTTTTCATATACTTTTTTTTGTGAAGAGGAGGAAATTTGAGTTAATATGACTATATATGGCTGTTGTGCCCTTCTCTTAGTGTTAATGAGTGCTTTTTATTTTTAGGTGCCATTAATCCTTTAGCTGAAGAATATCTGTCAAAGCAAGATCTACTTTTCTTAGACATGCTCAAGTTCTTGTGTTTGTGTGTAACTACTGCTCAGACCAATACTGTGTCCTTTAGGGCAGCTGATATTCGGAGGAAATTGTTAATGTTAATTGATTCTAGCACGCTAGAACCTACCAAATCCCTCCACCTGCATATGGTGAGTTACGTTAAATGAAGAAGCTCTTGGATTTTATCTGATGTTGCTGACTAAATGTAATGAGTTGACATGTAAGAATCACATGGTGTCTTTGAAGAATTGAAATTGCTTTCTTGAGAAATGAACCTGAGACTAGTTGGAAAATAACACTTTTAACGTGCTGTGAGCAAATTTAAGTGGATGCTGAAATATTAAAACTTAGATGGGTTTTGGATAGATTTATTCTCTTTAGGTTTAATTGTTATAAATTATTTTAATTAGGCTAAGAAACATAGACCCTGAATGTAAAAATATTACATAACATTGTGTTTGAAGTTGGAACCTTGCCTTTGACCTTAGTTTATATATTTTCAGTTTTTTGGATTTGATTTCAAAATGATAGTGAAGTTACAAGAATACTACAGTTAACGTTTTACCATATTTGCTTAATCATTCCCTCTCTCCACACACACACTCATACGCAGATACACACATCATTTTCTCTGAACCATTTTGAGATGAAATCGCACACGTGATGCTCCTTATACTTCAGTGTATATTTCCTAAAAACAAGGATACACTTCTGTTTAGCCACAGTATAATCATCAAAATCAGGAAGTTAACATTTTGTCATTGCATTTAGGTGTGTAGTCTGTTTAGTCTCCTTCAATCTGGAACAGTTCTTTTTTTTATTTATCATGACCTTCATATATTTGAATAGTACAGGTCCCTTGCTCTGAGTTTGTCTGATGTTTCCTTATGATTAAATTTATACTTTGACGAGAATGTTGCAAAAGTAATTCCTCTTCTGCCTTCACCCTGTCCATTTATCTGTCTGTTGATAGATATCTTCAAATCCAAGCCAACATCACACAGTTTATCTTTTTTTCTGTCCAAATTTATAAATTGCTTTACTAGCAGTTAAAAAAAACACCCCAAAACCTGACTATCCACTCAACCTGAATGCATTTACTTATTTGCTCAACAACCTTATATATAACTAATCTAACCATGTGGGCCATTTTCTTAGCCCTAGCCCCAGTGTATGTGGAGGTAGAGCCTCTGGCCTAGCGAGTAGCAAGTTGGGTCCTAGGTCATGTCCCCAGTTCCAGCAAATTGTCACACATGCCTGGTGAGTCCCTAATTAGAACCTCTGATCACAGTAAATTGTAGGCTGAGTCATTGGTGGAATCCCTGGTCATGTTTCCCACCCTTGGTGAAAATGCACGTCAATCCCTTAGAGTGAGAAGACAAGTTTGGTTGTGCCCTGTTTGGGCCCCTGGTTTAGGTCCTCGCAGATACTCAAGTCTCTATCCCTGACCAAGAGGAAGTTACATGCTTTTCTTGAAGTATTGATGTTTCAAGTCACTTCAGAACATTCAGTAACCTCATGTACATGACTCTCATGTCAAATAATGAAAAGGATCATTTAATTTCCTTTGTTAATATCTGAAAAGGATTAAGAAATATATTATTCTATATGTAAATAGGAAAAAAAATCCCTTATTATTTCAGTATAGCCAGAAAATTATTTGGTAGTTCTCGGAAACTGGCTGCCCAGATAATAACTACCATTATAACTGGTCGTTGCAGCAGCCCTTTCTGTGCATAGTACCATAAAGAGCCTTACAAGATTGCCTTATGGGAAGATAGGTCAAGGTTTTTGACTTCTTCCCTGCCCATGATTATCTCTGCTTTCCAAAATTCATTAATAGTTTTCAACCACTGGTCTATTATCCCCAAGCATTATTTTGGGAGAGTGGAGACTTACAGTTTCAGAATCTTGCTCAAGCTCTTAACTGCAACAGTGGTAATAATGATCATTTATTGAATTCCACAATAGAAGCTAGACTTTTACGTTTATTTTCTCTAATCCTCACAGTTATCTGGCCAGGAAAGTGATATATCTTCACTCTACTGATGAGGGTACGAAGGCCCTAGATGACATAAGGCAAGTTTTTTACCAGAAAAGTTAAGGCTGTGGTACTGGGATTTGAATTTTCTGTTTGACTTCAAAGCCTTTTGTTTTTACTATACACCTCTTAAATATTTACAGCTGTGTGTGTGTGTGTATGTGTGTGCATGTATGAATTTATGTTTATATACTTAAAAAAAATTTTTTTTTGAGACGGAATCTCGCTCTGTTGCCCGGGCTGGAGTGCAGTGGCGCGATCTCGGCTCACTGCAACCTTCGCCTCCCAGGTTCAAGCGATTCTCCTGCCTCAGCCTCCTGAGTAGCTGGGATTATAGGCGCCTGCCACCACCCCTGGCTAATTTTTGTGTTTTTAGTAGAGATGGAGTTTCACCATATTGGCCAGGCTGTTCTCAAACTCCTGACCTTGTGATCTGCCTGCTTCAGCCTCCCAAAGTGCTGGGATTACAGGTGTGAGCCACTGCACCCGGCCTATGTTTATATACTTTTTAAAGTAAATGATTTGTGGATAAACCTGATTTTTTTCCCTCCTACCATCTTAGTATCTAATGCTTTTAAAGGAGCTTCCTGGAGAAGAGTACCCCTTGCCAATGGAAGATGTTCTTGAACTTCTGAAACCACTATCGTAAGAAATTAAAACCTTATGTTATGTTCACTTTAAAGTTATAAAATAACTGATGTGTTCTGTTAAGCTTATAAAGTTGAACTTTTTTTTTTTTTTTACCACAGCAATGTGTGTTCTTTGTATCGTCGTGACCAAGATGTTTGTAAAACTATTTTAAACCATGTCCTTCATGTAGTGAAAAACCTAGGTCAAAGCAATATGGACTCTGAGAACACAAGGGATGCTCAAGGACAGTTTCTTACAGTAATTGGAGCATTTTGGTAGGTACAGTCTATTTTGTGGTCCTATTTTTCTTTTGCTATCTGTGGATACGAATGCAAGTTTTGTATCCACATCAGTGATTTCTTCTGATCTTCCTACATAGCTAATACATCTTTTAAGAATAGCAGAATGTAATTTGTGTTTCCCTCAGTCGCTTGAAGAACTACATTGCTTTTTGTTTAAGGCTTGGCTTTCTAAACCGTTTAACTCATTTCTCTGCAAATTTTGAATCACTTAATAATTTGACATGTATAGTGAATAAAGGATAACAAGCCAAAATCAGTTGAATTGTTCATGTTCCCTAAGTGAACAAATTGGCCTTGTCCTTATCTGGTTATTTTATGCATATTGTCACATTCCAAGATAACACCCTGATACTTACAGAGGAGCACATACATATAATTTAGAACCCTTGTACTATCTGATTGCTAAGTAATTCTAAATCAATAGACTGTTTTTTGTTTTTGTGTTTTTTTGAGATGGAGTCTCACTCCATCACCCAGGCTGCAGGCTGGAGTGCAGTGGTGCTATATCTGCTCACTGCAACCTCCGCCTAGTAGGTTCAAGCAATTGTCCTGCCTCAGCCTCCTGAGCAGCTATGATTACAGGCTCCCACCACAATGCCTGGCTAATTTTTTTGTATTTTTAGTAGAGATGGGGTTTCAACACGTTGGCCAGGCTGGTCCCAAACTCCTGACCTCAAGTGATCTGCCCACCTTGGCCTCCCAAAGTGTTGGGATTACAGGTGTGAGCCAGTGCACCCAGCCTGAATGTGGTTTTGAAATCTTCAATATACCAAACAAAGAAATCTTCAATATACCAAACAAAGAAATTTCTTTCTAAAGTAAGTTTACTGTAATGTAGTTTAGCCATTCTATGGTAGCCCCCAAAAAAGGACATAATGGTATAAAACAAATAATATACGCTAAAATGAATTCTTTTACACTAATTTCTTTTAGCTTGAATTTTTGGCAAGGTGAGTATGTTGGCATATTCCACATAATGACAAATAAGTTTAGCACAGAAAGACATATTGGAAGTAACTTACAATAACCTTTCAGTGAGTTTTCTGAGTGCTTTTATCAGAATGATTATTTAACTTTGGAAAACTTACTTGATTTCAGGCATCTAACAAAGGAGAGGAAATATATATTCTCTGTAAGAATGGCCCTAGTAAATTGCCTTAAAACTTTGCTTGAGGTGAGTTTTTGCATTTTTTTAGTAAGATCTCCATTGAAAATTTTAAAGCAGTCTTTGTTTGTTAATGAGTAATTTTTCTCTATTTCATATTTAACCACAGTTCTTTTCCCGTAGGCTGATCCTTATTCAAAATGGGCCATTCTTAATGTAATGGGAAAAGACTTTCCTGTAAATGAAGTATTTACACAATTTCTTGCTGACAATCATCACCAAGTTCGCATGTTGGCTGCAGAGTCAATCAATAGGTAATGGGTCAAATATTCATGAAGTATTTGGAATGCTGCAGATGGCAGTAGAATGTCTTACATAGTAACAGCTCACAGTTGCAATATTAAAAATAGCTAACACTTGTTGAGTATATACGGTGTGCCTGGCATTTATGTTTATTCTTAATTCTTATACTTCTGTCACTTAGATTCTATTATTTCCTTCAATTTATAAATGAAAACTGTGACTTAGTGAGGTAAAGTAACTTGCTTATGCCACAGAGCTAACAAGTGGTTGACTCGAGTGAGGTGTGTCTACTATCATGTACAGGAGATACGGACTGTTTGGAAATTACTGTGATTTTATGTGAGTTTATGAAATGATAATTAGCACATTTTACATTGTGCTAATTACAATTATATTTACTTGAGTGAACATCCAGTGAAGCACTGGAAGGATTCTATCATGAGGATCTTTTATATTTGAGTACTGGAATAAACATATATTAATTTCATTCTTTTTTTTTTTTTTTTTTTTTTTTTTTGAGATGGAGTTTCGCTCATGTCTCCCAGGCTGGAGTGCAATGGTGTGATCTCGGTTCACTTCAGCCTCTACCTCCTCGGTTCAAGCAATTCTCCTGCCTCATCCTCCTGAGTAGCTGGGATTACAGGCGCCTGCCACCATGCCCAGCTAATTTTTTTATTTTTCGTAGAGATGGAGTTTCACCATGTTGGCCAGGCTGGTCTTGAACTCCTGACCTCAGGTGATCCACCCACCTTGTCCTCCCAAAGTGCTGAGATTACAGGCGTGAGCCTTTAATCATGTAGTTTTTGTCATTCGTTCTGTTTATGTGATGGATTATGTTTATTGATTTGTGTATGCTGAATCAGCCTTGCGTCCCAGGGATGAAGCTGACTTGATAGTGGTGGATAAGCGTTTTGATGTGCTGGATTCAGTTTGCCAGTATTTTATCGAGGATTTTCACATCGATGTTCATCACGGATAATGGCCTGAAATTTTCTTTTTCTTGTGTCTCTGCCAGGTTTTGGAATCAGGATGATGCTGGCATCATAAAATGAGTTAGGGAAGAGTCCCTCTTTTTCTATTGTTTGGAATAGTTTCAGAAGGAATGGTACCAGCTCCTCTTTGTACCCCTGGTAGAATTCGGCTGTGAATCCCTCTGGTCCTGGGCTTTTTTTGGTTGGTAGGCTATTAATTACTGCCTCAATTTCAGAACTTGTTATTGGTCTATTCAGGGATTCGACTTCTTCCTGCTTTAGTTGGGAGGTTGTATGTGTCTAGGAATTTATCACTTTCTTCTAGATTTTCTAGTTTATTTGCATAGAGGTATTTATAGTATTCTCTGATGGTAGTTTGTATTTCTGTGGGATCAGTGGTATTATCCTCTTTATCATTTTTTATTGAGTCTATTTGATTCTTCTCTCTTTATTAGTCTTGCTAGCGGTCTGTTTTGTTAATCTTTTCAAAAAATCAGCTCCTGGATTAATTGATTTTTTTGAAAGGTTTTTTTTGTGTGTGTGTGTCTCTATCTCCTTCACTCAGCTCTGATCTTAGTTATTTCTTGTCTTCTGCTAGCTTTTGAATTTGTTTGCTCTTGCTTCTCTAGTTCTTTTAATTGTGATGTTAGGGTGTTGATTTTAGATCTTTCCCACTTTGTCCTGTGGGCATTTAGTGCTATAAATTTCCCTCTAAACACTGCTTTAGCTGTGTCCCAGAGATTCTGGTACATTGTGTCTTTGTTCTTATTGGTTTCAAAGAACTTACTTATTTCTGCCTTCATTTCATTATTTACCTAGTAGTCATTCAGGAGCAGGTTGTTCAGTTTCCATGTAGTTGTGTGGTTTTGAGTGAGTTTCTTAATCCTCAGTTCTAATTTGATTGCACTGTGATCTGAGAGACTGTTACGATTTCCATTCTTTTGCATTTGCTGAGGTTTGTTTTACTTTCAATTATGTGGTCAGTTTTAGAATAAGTGTGATGTGGTGCTGAGAAGAATGTATATTACTCTGTTGATTTGGGGTGGAGAGTTCTGTAGATGTCTGTTAGGTCTGCTTGGTCCAGAGTTCGGTTCAAGTCCTGAATATCCTTGTTAATTTTCTGTCTCATTGATCTGTCTAATATTGATGGTGGGGTGTTAAAGTCTCCCGCTATTATTGTGTGGAAGTCTAAGTCTCTTCGTAGGTCTAAGAACTTGCTTTATGAATCTGGGTGCTCCTGTATTGGGTGCATATATATTTAGGATAGTTAGCACTTCTTGTTGCATTGATCCTTATACCATCATGTAATGTCCTTCTTTGTCTTTCTTGATCTTTGTTGGATGTTTGTTTTATCAGAGACTAGGATTGCAACCCTTGCTTTTTTTCTTTCTATTTGCTTGGTAAATATTCCTCCACCCCTTTATTTTGAGCCTATGTGTGTCTTTGCATGTGAGATGGGTCCCCTGAATACAGCACACTGATGGGTCTTGACTCTTTATCCAATTTGCCAGTCTCTGTCTTTTAACTGGGACATTTTAGCCCATTTACATTTAAGGTTAATATTGTTATGTGTGAATTTGATCCTGTCATTATGATGCTAACTGGTTATTTTGCCCATTAGTTGATGCAGTTTCTTCATAGTGTCAATGGTCTTTACAATTTGGTATGTTTTTGCAGTGGCTGGTACCAGCTTTGCCTTTCCATATTTAGTGCTACCTTCAGGAGCTCTTGTAAGGCAGGCCTGGTGGTGACAAAATCTCTCAGCATTTGCT'

x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
y = np.mean([baseline[m].predict(x) for m in range(5)], axis=0)

q1 = y[0, :, 1]
q2 = y[0, :, 2]

In [9]:
(p2-q2).min()

-0.8292979

In [17]:
def new_spliceAI(chrom,pos,ref,alt,strand,max_distance,model=model_default,view_threshold=5,assembly='hg38',context=5000):

    start_time = time.time()
    
    start_index= pos-context-max_distance-1
    end_index= pos+context+max_distance+len(ref)-1
    chrom='chr'+str(chrom)
    
    dna_sequence=jkgenome.locus(chrom+':'+str(start_index)+'-'+str(end_index)+'+')
    dna_sequence=dna_sequence.twoBitFrag2('hg38').upper()
    
    print(len(dna_sequence))
    print(dna_sequence)
    if dna_sequence[pos-start_index-1:pos-start_index+len(ref)-1]!=ref:
        print('Error in positioning')
        print('The hg38 reference allele should be: '+dna_sequence[pos-start_index-1:pos-start_index+len(ref)-1])
        for i in range(-10,11):
            print(dna_sequence[pos-start_index+i-1],end="")
        print()
        print(' '*10+"-"*len(ref))
        return

    input_dna_sequence1,input_dna_sequence2=replace_dna_ref_to_alt2(dna_sequence,ref,alt,max_distance,context)
    print ("sequence changed --- %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    
    predicted_value1,_,_,_,_,_,_ = predict_sequence_with_context2(input_dna_sequence1,strand,model)
    predicted_value2,_,_,_,_,_,_ = predict_sequence_with_context2(input_dna_sequence2,strand,model)
    
    
    

In [65]:
spliceAI(8,140300616 ,'T','G','-',50,context=5000)[3]

Ref and Alt : (T to G)
CATTTTGTGTGGCCGGAGGCTAGCAGGAAGGTTCAATAGTTTCACATGCCTGTTGTTTCAAACAGAACACAAATACTTCAACTGTCTGGTTAGCGTGGTTT
                                                  -
CATTTTGTGTGGCCGGAGGCTAGCAGGAAGGTTCAATAGTTTCACATGCCGGTTGTTTCAAACAGAACACAAATACTTCAACTGTCTGGTTAGCGTGGTTT

sequence changed --- 0.0011761188507080078 seconds ---
prediction --- 0.331742525100708 seconds ---


,delta score(Donor Gain),pre-mRNA position
0,0.00026,-32
1,0.00003,34
2,0.00001,43
3,0.00001,-5
4,0.00001,-40


In [ ]:
def new_spliceAI(chrom,pos,ref,alt,strand,max_distance,model=model_default,view_threshold=5,assembly='hg38',context=5000):

    start_time = time.time()
    
    start_index= pos-context-max_distance-1
    end_index= pos+context+max_distance+len(ref)-1
    chrom='chr'+str(chrom)
    
    dna_sequence=jkgenome.locus(chrom+':'+str(start_index)+'-'+str(end_index)+'+')
    dna_sequence=dna_sequence.twoBitFrag2('hg38').upper()
    
    
    if dna_sequence[pos-start_index-1:pos-start_index+len(ref)-1]!=ref:
        print('Error in positioning')
        print('The hg38 reference allele should be: '+dna_sequence[pos-start_index-1:pos-start_index+len(ref)-1])
        for i in range(-10,11):
            print(dna_sequence[pos-start_index+i-1],end="")
        print()
        print(' '*10+"-"*len(ref))
        return
    
    
    input_dna_sequence1,input_dna_sequence2=replace_dna_ref_to_alt2(dna_sequence,ref,alt,max_distance,context)
    print ("sequence changed --- %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    
    predicted_value1,_,_,_,_,_,_ = predict_sequence_with_context2(input_dna_sequence1,strand,model)
    predicted_value2,_,_,_,_,_,_ = predict_sequence_with_context2(input_dna_sequence2,strand,model)
    
    print(input_dna_sequence1)
    print(input_dna_sequence2)
    
    if strand == '-':
        predicted_value1 = predicted_value1[:, ::-1, ::1]
        predicted_value2 = predicted_value2[:, ::-1, ::1]
    
    print ("prediction --- %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    
    predicted_value_for_common1=np.concatenate((predicted_value1[0][:max_distance],predicted_value1[0][-max_distance:]))
    predicted_value_for_common2=np.concatenate((predicted_value2[0][:max_distance],predicted_value2[0][-max_distance:]))
    
    
    predicted_value_for_raw1=predicted_value1[0][max_distance:max_distance+len(ref)]
    predicted_value_for_raw2=predicted_value2[0][max_distance:max_distance+len(alt)]
    
    acceptor_prob1 = predicted_value_for_common1[ :, 1]
    donor_prob1 = predicted_value_for_common1[ :, 2]
    
    acceptor_prob2 = predicted_value_for_common2[ :, 1]
    donor_prob2 = predicted_value_for_common2[ :, 2]
    

    
    delta_acceptor_prob = acceptor_prob1 - acceptor_prob2
    delta_donor_prob = donor_prob1 - donor_prob2
    
    delta_acceptor_sorted_index = (acceptor_prob1 - acceptor_prob2).argsort()
    delta_acceptor_sorted_index = np.flip(delta_acceptor_sorted_index)
    
    delta_donor_sorted_index = (donor_prob1 - donor_prob2).argsort()
    delta_donor_sorted_index = np.flip(delta_donor_sorted_index)
    
    delta_acceptor_sorted_value = delta_acceptor_prob[delta_acceptor_sorted_index]
    delta_donor_sorted_value = delta_donor_prob[delta_donor_sorted_index]

    
    df_acceptor_loss,df_donor_loss,df_acceptor_gain,df_donor_gain,\
    df_raw_acceptor_loss,df_raw_donor_loss,df_raw_acceptor_gain,df_raw_donor_gain \
    = view_result2(  (delta_acceptor_prob, delta_donor_prob, delta_acceptor_sorted_index, delta_donor_sorted_index, \
                    delta_acceptor_sorted_value, delta_donor_sorted_value, \
                    predicted_value_for_raw1, predicted_value_for_raw2),view_threshold,max_distance)
    
    delta_common=pd.concat([df_acceptor_loss,df_donor_loss,df_acceptor_gain,df_donor_gain], axis=1)


    return df_acceptor_loss,df_donor_loss,df_acceptor_gain,df_donor_gain,\
            df_raw_acceptor_loss,df_raw_donor_loss,df_raw_acceptor_gain,df_raw_donor_gain,predicted_value_for_common1,predicted_value_for_common2

In [95]:
# compare_models(8,140300615,'T','G','-',3000,model1=model_default,model2=model_default,context1=5000,context2=5000)[0]

In [19]:
def predict_sequence_with_context2(dna_sequence,strand,model,assembly='hg38'):
    encoded_dna_sequence = one_hot_encode(dna_sequence)[None, :]
    
    if strand == '-':
        encoded_dna_sequence = encoded_dna_sequence[:, ::-1, ::-1]
    

    predicted_value = np.mean([model[m].predict(encoded_dna_sequence) for m in range(len(model))], axis=0)
    
    
    acceptor_prob = predicted_value[0, :, 1]
    donor_prob = predicted_value[0, :, 2]

    acceptor_prob_sorted_index=acceptor_prob.argsort()
    acceptor_prob_sorted_index=np.flip(acceptor_prob_sorted_index)
    
    donor_prob_sorted_index=donor_prob.argsort()
    donor_prob_sorted_index=np.flip(donor_prob_sorted_index)
    
    acceptor_prob_sorted_value=acceptor_prob[acceptor_prob_sorted_index]
    donor_prob_sorted_value=donor_prob[donor_prob_sorted_index]
    
    #                 0          1             2                 3                          4
    return predicted_value, acceptor_prob, donor_prob, acceptor_prob_sorted_index, donor_prob_sorted_index,\
        acceptor_prob_sorted_value, donor_prob_sorted_value

In [20]:
def replace_dna_ref_to_alt2(dna_sequence,ref,alt,max_distance,context):    
    original_dna_sequence=dna_sequence
    altered_dna_sequence=dna_sequence[:context+max_distance] + alt + dna_sequence[-context-max_distance:]

    
    if max(len(ref),len(alt))<30:
        print("Ref and Alt : ("+ref+" to "+alt+")")
        if len(ref)>len(alt):
            print(original_dna_sequence[context+max_distance-50:context+max_distance]+ref+                         original_dna_sequence[context+max_distance+len(ref):context+max_distance+len(ref)+50])
            print(" "*50+"-"*len(ref))
            print(altered_dna_sequence[context+max_distance-50:context+max_distance]+alt+'.'*(len(ref)-len(alt))+altered_dna_sequence[context+max_distance+len(alt):context+max_distance+len(alt)+50])


        else:
            print(original_dna_sequence[context+max_distance-50:context+max_distance]+ref+'.'*(len(alt)-len(ref))+original_dna_sequence[context+max_distance+len(ref):context+max_distance+len(ref)+50])
            print(" "*50+"-"*len(alt))
            print(altered_dna_sequence[context+max_distance-50:context+max_distance]+alt                        +altered_dna_sequence[context+max_distance+len(alt):context+max_distance+len(alt)+50])
        print()
    return original_dna_sequence,altered_dna_sequence

In [21]:
def view_result2(return_above,view_threshold,max_distance):
    (delta_acceptor_prob, delta_donor_prob, delta_acceptor_sorted_index, delta_donor_sorted_index, \
    delta_acceptor_sorted_value, delta_donor_sorted_value, predicted_value_for_raw1, predicted_value_for_raw2) = return_above
    
    count_acceptor_positive=0
    count_donor_positive=0
    for i in delta_acceptor_sorted_value:
        if i>=0:
            count_acceptor_positive+=1
    
    for i in delta_donor_sorted_value:
        if i>=0:
            count_donor_positive+=1
    
    delta_acceptor_sorted_index_copied=delta_acceptor_sorted_index.copy()
    delta_donor_sorted_index_copied=delta_donor_sorted_index.copy()
    
    for i in range(2*max_distance):
        if delta_acceptor_sorted_index_copied[i]>=max_distance:
            delta_acceptor_sorted_index_copied[i]+=1
        if delta_donor_sorted_index_copied[i]>=max_distance:
            delta_donor_sorted_index_copied[i]+=1
    
  
    df_acceptor_loss=pd.DataFrame()
    
    df_acceptor_loss['delta score(Acceptor Loss)']=delta_acceptor_sorted_value[:min(view_threshold,count_acceptor_positive)]
    df_acceptor_loss['pre-mRNA position']=(delta_acceptor_sorted_index_copied-max_distance)[:min(view_threshold,count_acceptor_positive)]
    
    
    
    
    delta_acceptor_sorted_value_r = delta_acceptor_sorted_value[::-1]
    delta_acceptor_sorted_index_r = delta_acceptor_sorted_index_copied[::-1]
    
    df_acceptor_gain=pd.DataFrame()
    
    df_acceptor_gain['delta score(Acceptor Gain)']=-delta_acceptor_sorted_value_r[:min(view_threshold,2*max_distance-count_acceptor_positive)]
    df_acceptor_gain['pre-mRNA position']=(delta_acceptor_sorted_index_r-max_distance)[:min(view_threshold,2*max_distance-count_acceptor_positive)]
    
    
    
    
    df_donor_loss=pd.DataFrame()
    
    df_donor_loss['delta score(Donor Loss)']=delta_donor_sorted_value[:min(view_threshold,count_donor_positive)]
    df_donor_loss['pre-mRNA position']=(delta_donor_sorted_index_copied-max_distance)[:min(view_threshold,count_donor_positive)]
    
    
    
    
    delta_donor_sorted_value_r = delta_donor_sorted_value[::-1]
    delta_donor_sorted_index_r = delta_donor_sorted_index_copied[::-1]
    
    df_donor_gain=pd.DataFrame()
    
    df_donor_gain['delta score(Donor Gain)']=-delta_donor_sorted_value_r[:min(view_threshold,2*max_distance-count_donor_positive)]
    df_donor_gain['pre-mRNA position']=(delta_donor_sorted_index_r-max_distance)[:min(view_threshold,2*max_distance-count_donor_positive)]
    
    
    
    ## START FOR RAW ##
    
    delta_in_raw = predicted_value_for_raw1-predicted_value_for_raw2
    delta_raw_acceptor= delta_in_raw[:, 1]
    delta_raw_donor= delta_in_raw[:, 2]
    
    
    count_delta_acceptor_positive=0
    count_delta_donor_positive=0
    for i in delta_raw_acceptor:
        if i>=0:
            count_delta_acceptor_positive+=1
    
    for i in delta_raw_donor:
        if i>=0:
            count_delta_donor_positive+=1
    

    
    delta_raw_acceptor_sorted_index_copied=delta_raw_acceptor.argsort().copy()
    delta_raw_donor_sorted_index_copied=delta_raw_donor.argsort().copy()
    
    delta_raw_acceptor_sorted_index_copied = np.flip(delta_raw_acceptor_sorted_index_copied)
    delta_raw_donor_sorted_index_copied = np.flip(delta_raw_donor_sorted_index_copied)
    
    delta_raw_acceptor_sorted_value=delta_raw_acceptor[delta_raw_acceptor_sorted_index_copied]
    delta_raw_donor_sorted_value=delta_raw_donor[delta_raw_donor_sorted_index_copied]
    
    
    
    for i in range(len(delta_raw_acceptor)):
        delta_raw_acceptor_sorted_index_copied[i]+=1
        delta_raw_donor_sorted_index_copied[i]+=1
    

    
  
    df_raw_acceptor_loss=pd.DataFrame()
    
    df_raw_acceptor_loss['delta score in raw(Acceptor Loss)']=delta_raw_acceptor_sorted_value[:min(view_threshold,count_delta_acceptor_positive)]
    df_raw_acceptor_loss['pre-mRNA position']=delta_raw_acceptor_sorted_index_copied[:min(view_threshold,count_delta_acceptor_positive)]
    
    
    
    
    delta_raw_acceptor_sorted_value_r = delta_raw_acceptor_sorted_value[::-1]
    delta_raw_acceptor_sorted_index_r = delta_raw_acceptor_sorted_index_copied[::-1]
    
    df_raw_acceptor_gain=pd.DataFrame()
    
    df_raw_acceptor_gain['delta score in raw(Acceptor Gain)']=-delta_raw_acceptor_sorted_value_r[:min(view_threshold,len(delta_raw_acceptor_sorted_value_r)-count_delta_acceptor_positive)]
    df_raw_acceptor_gain['pre-mRNA position']=delta_raw_acceptor_sorted_index_r[:min(view_threshold,len(delta_raw_acceptor_sorted_value_r)-count_delta_acceptor_positive)]
    
    
    
    df_raw_donor_loss=pd.DataFrame()
    
    df_raw_donor_loss['delta score in raw(Donor Loss)']=delta_raw_donor_sorted_value[:min(view_threshold,count_delta_donor_positive)]
    df_raw_donor_loss['pre-mRNA position']=delta_raw_donor_sorted_index_copied[:min(view_threshold,count_delta_donor_positive)]
    
    
    
    
    delta_raw_donor_sorted_value_r = delta_raw_donor_sorted_value[::-1]
    delta_raw_donor_sorted_index_r = delta_raw_donor_sorted_index_copied[::-1]
    
    df_raw_donor_gain=pd.DataFrame()
    
    df_raw_donor_gain['delta score in raw(Donor Gain)']=-delta_raw_donor_sorted_value_r[:min(view_threshold,len(delta_raw_donor_sorted_value_r)-count_delta_donor_positive)]
    df_raw_donor_gain['pre-mRNA position']=delta_raw_donor_sorted_index_r[:min(view_threshold,len(delta_raw_donor_sorted_value_r)-count_delta_donor_positive)]
    
    
    
    pd.options.display.float_format = '{:.5f}'.format
    
    return df_acceptor_loss,df_donor_loss,df_acceptor_gain,df_donor_gain,\
            df_raw_acceptor_loss,df_raw_donor_loss,df_raw_acceptor_gain,df_raw_donor_gain

In [22]:
def spliceAI(chrom,pos,ref,alt,strand,max_distance,model=baseline,view_threshold=5,assembly='hg38',context=5000):

    start_time = time.time()
    
    start_index= pos-context-max_distance-1
    end_index= pos+context+max_distance+len(ref)-1
    chrom='chr'+str(chrom)
    
    dna_sequence=jkgenome.locus(chrom+':'+str(start_index)+'-'+str(end_index)+'+')
    dna_sequence=dna_sequence.twoBitFrag2('hg38').upper()
    
    
    if dna_sequence[pos-start_index-1:pos-start_index+len(ref)-1]!=ref:
        print('Error in positioning')
        print('The hg38 reference allele should be: '+dna_sequence[pos-start_index-1:pos-start_index+len(ref)-1])
        for i in range(-10,11):
            print(dna_sequence[pos-start_index+i-1],end="")
        print()
        print(' '*10+"-"*len(ref))
        return
    
    
    input_dna_sequence1,input_dna_sequence2=replace_dna_ref_to_alt2(dna_sequence,ref,alt,max_distance,context)
    print ("sequence changed --- %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    
    predicted_value1,_,_,_,_,_,_ = predict_sequence_with_context2(input_dna_sequence1,strand,model)
    predicted_value2,_,_,_,_,_,_ = predict_sequence_with_context2(input_dna_sequence2,strand,model)
    
    if strand == '-':
        predicted_value1 = predicted_value1[:, ::-1, ::1]
        predicted_value2 = predicted_value2[:, ::-1, ::1]
    
    print ("prediction --- %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    
    predicted_value_for_common1=np.concatenate((predicted_value1[0][:max_distance],predicted_value1[0][-max_distance:]))
    predicted_value_for_common2=np.concatenate((predicted_value2[0][:max_distance],predicted_value2[0][-max_distance:]))
    
    
    predicted_value_for_raw1=predicted_value1[0][max_distance:max_distance+len(ref)]
    predicted_value_for_raw2=predicted_value2[0][max_distance:max_distance+len(alt)]
    
    acceptor_prob1 = predicted_value_for_common1[ :, 1]
    donor_prob1 = predicted_value_for_common1[ :, 2]
    
    acceptor_prob2 = predicted_value_for_common2[ :, 1]
    donor_prob2 = predicted_value_for_common2[ :, 2]
    

    
    delta_acceptor_prob = acceptor_prob1 - acceptor_prob2
    delta_donor_prob = donor_prob1 - donor_prob2
    
    delta_acceptor_sorted_index = (acceptor_prob1 - acceptor_prob2).argsort()
    delta_acceptor_sorted_index = np.flip(delta_acceptor_sorted_index)
    
    delta_donor_sorted_index = (donor_prob1 - donor_prob2).argsort()
    delta_donor_sorted_index = np.flip(delta_donor_sorted_index)
    
    delta_acceptor_sorted_value = delta_acceptor_prob[delta_acceptor_sorted_index]
    delta_donor_sorted_value = delta_donor_prob[delta_donor_sorted_index]

    
    df_acceptor_loss,df_donor_loss,df_acceptor_gain,df_donor_gain,\
    df_raw_acceptor_loss,df_raw_donor_loss,df_raw_acceptor_gain,df_raw_donor_gain \
    = view_result2(  (delta_acceptor_prob, delta_donor_prob, delta_acceptor_sorted_index, delta_donor_sorted_index, \
                    delta_acceptor_sorted_value, delta_donor_sorted_value, \
                    predicted_value_for_raw1, predicted_value_for_raw2),view_threshold,max_distance)
    
    delta_common=pd.concat([df_acceptor_loss,df_donor_loss,df_acceptor_gain,df_donor_gain], axis=1)


    return df_acceptor_loss,df_donor_loss,df_acceptor_gain,df_donor_gain,\
            df_raw_acceptor_loss,df_raw_donor_loss,df_raw_acceptor_gain,df_raw_donor_gain,predicted_value_for_common1,predicted_value_for_common2

In [23]:
def compare_models(chrom,pos,ref,alt,strand,max_distance,model1=model_default,
                  model2=model_reg,view_threshold=5,assembly='hg38',context1=5000,context2=5000):
    sresult1=spliceAI(chrom,pos,ref,alt,strand,max_distance,model=model1,context=context1)
    sresult2=spliceAI(chrom,pos,ref,alt,strand,max_distance,model=model2,context=context2)
    view_common=[]
    for i in range(8):
        view_common.append(pd.concat([sresult1[i],sresult2[i]], axis=1))
    return view_common

In [69]:
xxx=np.array([1,2,3,4,5])
np.flip(xxx)

array([5, 4, 3, 2, 1])

In [117]:
compare_models(11,108299060,'G','GGCTCTACCCTTTGAAAGAAGGGCTCTCAAAAAATTTGTGAACATATTTTAGTAAATACCATCATGATGGAAAGAGCACGAAGAGCCCTCTGGGTTGCTGAAAATGTCCTGTCTTCATCTGGTTAGGGAGAACACAGGCAGAAAAAAATGTGTATTTGTATGTATAAATTTTCATTGAACTGTGTGTTTAAGATTATGGAATTTGTGTGATACCTCAACAAAAATTTTAACAGTTTTAAGGTTGATTCTCTCTCTCTTTTTTTTTTTTTCTTTTGTCACCCAGGCTAGAGTGCAGTGGTGCCATCTTGGCTCACTGCAGCCTCCGCCTCCTGGGTTCAAGTGATTCTCCTGCCTCAGCCTCCTGAGTAGCTGGGATTACGGGC','+',10000,model1=model_default,
                  model2=model_default,context2=400)[6]

sequence changed --- 0.005728006362915039 seconds ---
prediction --- 0.08041763305664062 seconds ---
YAHAHAH
[1.5004422e-06 8.1803819e-06 1.7104522e-06 ... 4.5079382e-08 5.4666583e-07
 5.0964251e-09]
sequence changed --- 0.0021681785583496094 seconds ---
prediction --- 0.05929374694824219 seconds ---
YAHAHAH
[7.4787740e-09 8.2853923e-08 9.0118746e-08 ... 1.4736975e-06 1.4884245e-08
 5.1022521e-08]


,delta score in raw(Acceptor Gain),pre-mRNA position,delta score in raw(Acceptor Gain),pre-mRNA position
0,0.00008,376,0.00167,253
1,0.00003,131,0.00047,156
2,0.00002,233,0.00013,327
3,0.00002,244,0.00003,219
4,0.00001,36,0.00003,357


In [27]:
spliceAI(8,140300616,'T','G','-',10000)[1]

Ref and Alt : (T to G)
CATTTTGTGTGGCCGGAGGCTAGCAGGAAGGTTCAATAGTTTCACATGCCTGTTGTTTCAAACAGAACACAAATACTTCAACTGTCTGGTTAGCGTGGTTT
                                                  -
CATTTTGTGTGGCCGGAGGCTAGCAGGAAGGTTCAATAGTTTCACATGCCGGTTGTTTCAAACAGAACACAAATACTTCAACTGTCTGGTTAGCGTGGTTT

sequence changed --- 0.006513833999633789 seconds ---
prediction --- 0.3221776485443115 seconds ---


,delta score(Donor Loss),pre-mRNA position
0,0.48042,-147
1,0.16355,311
2,0.05391,833
3,0.05093,2069
4,0.02090,-3497


In [129]:
compare_models(11,108270482,'GGTAA','G','+',10000)[2]

Ref and Alt : (GGTAA to G)
AAGCTAGACTTTTACGTTTATTTTCTCTAATCCTCACAGTTATCTGGCCAGGTAAGTGATATATCTTCACTCTACTGATGAGGGTACGAAGGCCCTAGATGACAT
                                                  -----
AAGCTAGACTTTTACGTTTATTTTCTCTAATCCTCACAGTTATCTGGCCAG....GTGATATATCTTCACTCTACTGATGAGGGTACGAAGGCCCTAGATGACAT

sequence changed --- 0.0057561397552490234 seconds ---
prediction --- 0.07665705680847168 seconds ---
YAHAHAH
0.9867251
0.9884588
0.9867251
0.9884588
Ref and Alt : (GGTAA to G)
AAGCTAGACTTTTACGTTTATTTTCTCTAATCCTCACAGTTATCTGGCCAGGTAAGTGATATATCTTCACTCTACTGATGAGGGTACGAAGGCCCTAGATGACAT
                                                  -----
AAGCTAGACTTTTACGTTTATTTTCTCTAATCCTCACAGTTATCTGGCCAG....GTGATATATCTTCACTCTACTGATGAGGGTACGAAGGCCCTAGATGACAT

sequence changed --- 0.003066539764404297 seconds ---
prediction --- 0.05665874481201172 seconds ---
YAHAHAH
0.098378174
0.9413468
0.098378174
0.9413468


,delta score(Acceptor Gain),pre-mRNA position,delta score(Acceptor Gain),pre-mRNA position
